In [58]:
import cv2
import numpy as np

from keras.models import load_model
from keras.preprocessing.image import img_to_array
from playsound import playsound
from threading import Thread
from keras.models import model_from_json

In [59]:
def start_alarm(sound):
    """Play the alarm sound"""
    playsound('data/alarm.mp3')

In [60]:
# 0-yawn, 1-no_yawn, 2-Closed, 3-Open
classes = ['Yawn', 'No_Yawn', 'Closed', 'Open']
face_cascade = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
left_eye_cascade = cv2.CascadeClassifier("data/haarcascade_lefteye_2splits.xml")
right_eye_cascade = cv2.CascadeClassifier("data/haarcascade_righteye_2splits.xml")

In [61]:
cap = cv2.VideoCapture(0)


In [62]:
# Load the trained model
json_file = open('yawn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
yawn_model = model_from_json(loaded_model_json)

yawn_model.load_weights("yawn_detection_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [63]:
count = 0
alarm_on = False
alarm_sound = "data/alarm.mp3"
# status1 = ''
# status2 = ''

left_eye_status = -1
right_eye_status = -1
mouth_status = -1


In [64]:
while True:
    _, frame = cap.read()
    height = frame.shape[0]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 1)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Process the entire face for yawn detection
        face_img = roi_color.copy()
        face_img = cv2.resize(face_img, (145, 145))
        face_img = face_img.astype('float') / 255.0
        face_img = img_to_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        face_pred = yawn_model.predict(face_img, verbose=0)
        yawn_status = np.argmax(face_pred)
        
        # Process left eye
        left_eye = left_eye_cascade.detectMultiScale(roi_gray)
        for (x1, y1, w1, h1) in left_eye:
            cv2.rectangle(roi_color, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 1)
            eye1 = roi_color[y1:y1+h1, x1:x1+w1]
            eye1 = cv2.resize(eye1, (145, 145))
            eye1 = eye1.astype('float') / 255.0
            eye1 = img_to_array(eye1)
            eye1 = np.expand_dims(eye1, axis=0)
            pred1 = yawn_model.predict(eye1,verbose=0)
            left_eye_status = np.argmax(pred1)
            
            # print(status1)
            #status1 = classes[pred1.argmax(axis=-1)[0]]
            break
        
        # Process right eye
        right_eye = right_eye_cascade.detectMultiScale(roi_gray)
        for (x2, y2, w2, h2) in right_eye:
            cv2.rectangle(roi_color, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 1)
            eye2 = roi_color[y2:y2 + h2, x2:x2 + w2]
            eye2 = cv2.resize(eye2, (145, 145))
            eye2 = eye2.astype('float') / 255.0
            eye2 = img_to_array(eye2)
            eye2 = np.expand_dims(eye2, axis=0)
            pred2 = yawn_model.predict(eye2,verbose=0)
            right_eye_status = np.argmax(pred2)
            # print(status2)
            #status2 = classes[pred2.argmax(axis=-1)[0]]
            break
        
        
        # # Mouth/Yawn detection - focus on the lower part of the face
        # mouth_y = y + int(h * 0.6)  # Start from 60% down the face
        # mouth_h = int(h * 0.4)      # Use the bottom 40% of the face
        
        # if mouth_y + mouth_h <= frame.shape[0]:  # Ensure we don't go out of bounds
        #     mouth_roi = roi_color[int(h*0.6):h, 0:w]  # Extract the mouth region
            
        #     if mouth_roi.size > 0:  # Check if the ROI is valid
        #         mouth_roi = cv2.resize(mouth_roi, (145, 145))
        #         mouth_roi = mouth_roi.astype('float') / 255.0
        #         mouth_roi = img_to_array(mouth_roi)
        #         mouth_roi = np.expand_dims(mouth_roi, axis=0)
        #         yawn_pred = yawn_model.predict(mouth_roi, verbose=0)
        #         mouth_status = np.argmax(yawn_pred)
                
        #         # Draw mouth/yawn region
        #         cv2.rectangle(roi_color, (0, int(h*0.6)), (w, h), (0, 0, 255), 1)
        
        
        # If the eyes are closed, start counting
        if left_eye_status == 2 and right_eye_status == 2:
        #if pred1 == 2 and pred2 == 2:
            count += 1
            cv2.putText(frame, "Eyes Closed, Frame count: " + str(count), (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1)
            # if eyes are closed for 10 consecutive frames, start the alarm
            if count >= 7:
                cv2.putText(frame, "Drowsiness Alert!!!", (100, height-40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                if not alarm_on:
                    alarm_on = True
                    # play the alarm sound in a new thread
                    t = Thread(target=start_alarm, args=(alarm_sound,))
                    t.daemon = True
                    t.start()
        else:
            cv2.putText(frame, "Eyes Open", (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            count = 0
            alarm_on = False
            
        
        
                # Check for yawning condition (class index 0)
        if yawn_status == 0:  # Yawning detected
            yawn_count += 1
            cv2.putText(frame, "Yawning Detected, Count: " + str(yawn_count), (10, 60), 
                        cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1)
            
            if yawn_count >= 7:
                cv2.putText(frame, "ALERT! Driver Drowsy!", (100, height-20), 
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                if not alarm_on:
                    alarm_on = True
                    t = Thread(target=start_alarm, args=(alarm_sound,))
                    t.daemon = True
                    t.start()
        else:
            cv2.putText(frame, "No Yawning", (10, 60), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 1)
            yawn_count = 0
        
        
        # # Check for yawning condition (class index 0)
        # if mouth_status == 0:  # Yawning detected
        #     yawn_count += 1
        #     cv2.putText(frame, "Yawning Detected, Count: " + str(yawn_count), (10, 60), 
        #                 cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1)
            
        #     if yawn_count >= 7:
        #         cv2.putText(frame, "ALERT! Driver Drowsy!", (100, height-20), 
        #                 cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        #         if not alarm_on:
        #             alarm_on = True
        #             t = Thread(target=start_alarm, args=(alarm_sound,))
        #             t.daemon = True
        #             t.start()
        # else:
        #     cv2.putText(frame, "No Yawning", (10, 60), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 1)
        #     yawn_count = 0
        

        # Display the class predictions (helpful for debugging)
        cv2.putText(frame, f"Left Eye: {classes[left_eye_status] if left_eye_status != -1 else 'None'}", (10, 90), 
                cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
        cv2.putText(frame, f"Right Eye: {classes[right_eye_status] if right_eye_status != -1 else 'None'}", (10, 120), 
                cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
        cv2.putText(frame, f"Face/Yawn: {classes[yawn_status] if yawn_status != -1 else 'None'}", (10, 150), 
                cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
        
        # Reset alarm if both conditions are normal
        if (left_eye_status != 2 or right_eye_status != 2) and mouth_status != 0:
            alarm_on = False
        
    cv2.imshow("Drowsiness Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


    Error 259 for command:
        play data/alarm.mp3 wait
    The driver cannot recognize the specified command parameter.
Exception in thread Thread-195 (start_alarm):
Traceback (most recent call last):
  File "c:\Users\ahmedhoss\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\ahmedhoss\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmedhoss\AppData\Local\Temp\ipykernel_8928\3586823210.py", line 3, in start_alarm
  File "c:\Users\ahmedhoss\anaconda3\Lib\site-packages\playsound.py", line 73, in _playsoundWin
    winCommand(u'play {}{}'.format(sound, ' wait' if block else ''))
  File "c:\Users\ahmedhoss\anaconda3\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 259 for command:
        play data/alarm.mp3 wait
    The driver cannot recognize the specified command parameter.

    Error